In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Setting up the configs

In [9]:
TM_LEAGUES = {
    'ENG-Premier League': {'url_name': 'premier-league', 'id': 'GB1'},
    'ESP-La Liga': {'url_name': 'laliga', 'id': 'ES1'},
    'GER-Bundesliga': {'url_name': 'bundesliga', 'id': 'L1'},
    'ITA-Serie A': {'url_name': 'serie-a', 'id': 'IT1'},
    'FRA-Ligue 1': {'url_name': 'ligue-1', 'id': 'FR1'}
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9'
}

In [10]:
league_info= TM_LEAGUES['ENG-Premier League']
year="2017"
url = f"https://www.transfermarkt.com/{league_info['url_name']}/transfers/wettbewerb/{league_info['id']}/plus/?saison_id={year}"
response = requests.get(url,headers=HEADERS)
soup=BeautifulSoup(response.text,'html.parser')
print(url)


https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2017


In [ ]:
boxes = soup.find_all("div", class_="box")
club_boxes= boxes[3:23]
curr_club= club_boxes[0]
h2 = curr_club.find("h2", class_="content-box-headline")
club_name = curr_club.select_one("h2 a[title]").get('title')
print(club_name)
print(curr_club)

Chelsea FC
<div class="box">
<h2 class="content-box-headline content-box-headline--inverted content-box-headline--logo" id="to-631"><a href="/fc-chelsea/transfers/verein/631/saison_id/2017" title="Chelsea FC"><img alt="Chelsea FC" class="" src="https://tmssl.akamaized.net//images/wappen/homepageSmall/631.png?lm=1682435911" title="Chelsea FC"/></a> <a href="/fc-chelsea/transfers/verein/631/saison_id/2017" title="Chelsea FCArray">Chelsea FC</a> </h2>
<div class="responsive-table">
<table>
<thead>
<tr>
<th class="spieler-transfer-cell">In</th>
<th class="zentriert alter-transfer-cell">Age</th>
<th class="zentriert nat-transfer-cell">Nat.</th>
<th class="pos-transfer-cell">Position</th>
<th class="kurzpos-transfer-cell zentriert">Pos</th>
<th class="rechts mw-transfer-cell">Market value</th>
<th class="verein-transfer-cell" colspan="2">Left</th>
<th class="rechts abloese-transfer-cell">Fee</th>
</tr>
</thead>
<tbody>
<tr>
<td><div class="di nowrap"><span class="hide-for-small"><a href="/al

In [12]:
all_transfers=[]
transfer_tables= curr_club.find_all('table') #change to find_all for the final script
transfer_table=transfer_tables[0]
thead= transfer_table.find('thead')
header_cells = thead.find_all("th")
headers = [th.get_text(strip=True) for th in header_cells]
headers.remove('Position')
headers[0]='Player'
headers.append('In')
direction ='unknown'
if 'Left' in headers:
    direction= 'In'
else :
    direction='Out'
print(headers)
print(transfer_tables)

['Player', 'Age', 'Nat.', 'Pos', 'Market value', 'Left', 'Fee', 'In']
[<table>
<thead>
<tr>
<th class="spieler-transfer-cell">In</th>
<th class="zentriert alter-transfer-cell">Age</th>
<th class="zentriert nat-transfer-cell">Nat.</th>
<th class="pos-transfer-cell">Position</th>
<th class="kurzpos-transfer-cell zentriert">Pos</th>
<th class="rechts mw-transfer-cell">Market value</th>
<th class="verein-transfer-cell" colspan="2">Left</th>
<th class="rechts abloese-transfer-cell">Fee</th>
</tr>
</thead>
<tbody>
<tr>
<td><div class="di nowrap"><span class="hide-for-small"><a href="/alvaro-morata/profil/spieler/128223" title="Álvaro Morata">Álvaro Morata</a></span></div><div class="di nowrap"><span class="show-for-small"><a href="/alvaro-morata/profil/spieler/128223" title="Álvaro Morata">Á. Morata</a></span></div></td>
<td class="zentriert alter-transfer-cell">24</td>
<td class="zentriert nat-transfer-cell"><img alt="Spain" class="flaggenrahmen" src="https://tmssl.akamaized.net//images/fla

In [13]:
tbody=transfer_table.find('tbody')
trs= tbody.find_all('tr')
print(len(trs))
# print(tds)
for tr in trs:
    # print(td.get_text(separator='/',strip=True))
    player_str=tr.get_text(separator='/',strip=True)
    player_list= player_str.split(sep='/')
    # print(player_list)
    player_name= player_list[0]
    player_mv=player_list[5]
    player_club= player_list[6]
    player_fee= player_list[7]
    all_transfers.append({
                            "League": "EPL",
                            "Season":year+ '-'+str(int(year[2:])+1),
                            "Club": club_name,
                            "Direction": direction,
                            "Player": player_name,
                            "Market_Value_Raw": player_mv,
                            "Counterpart_Club": player_club,
                            "Fee": player_fee,
                            # "Fee_Cleaned": clean_money(fee_raw),
                            # "Market_Value_Cleaned": clean_money(market_val_raw)
                        })
    # print(td.find('img').get('title'))
# print(tbody)


40


In [14]:
pd.DataFrame(all_transfers)

,League,Season,Club,Direction,Player,Market_Value_Raw,Counterpart_Club,Fee
0,EPL,2017-18,Chelsea FC,In,Álvaro Morata,€40.00m,Real Madrid,€66.00m
1,EPL,2017-18,Chelsea FC,In,Tiemoué Bakayoko,€16.00m,Monaco,€40.00m
2,EPL,2017-18,Chelsea FC,In,Danny Drinkwater,€9.00m,Leicester,€37.90m
3,EPL,2017-18,Chelsea FC,In,Antonio Rüdiger,€25.00m,Roma,€35.00m
4,EPL,2017-18,Chelsea FC,In,Davide Zappacosta,€8.50m,Torino,€25.00m
5,EPL,2017-18,Chelsea FC,In,Emerson,€14.00m,Roma,€20.00m
6,EPL,2017-18,Chelsea FC,In,Olivier Giroud,€20.00m,Arsenal,€17.00m
7,EPL,2017-18,Chelsea FC,In,Ross Barkley,€25.00m,Everton,€16.80m
8,EPL,2017-18,Chelsea FC,In,Ethan Ampadu,-,Exeter City,€2.80m
9,EPL,2017-18,Chelsea FC,In,Willy Caballero,€1.50m,Man City,free transfer
